In [1]:
import sys

sys.path.append('../')

In [2]:
import os
from glob import glob
import yaml
from easydict import EasyDict as edict
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import torch
import torch.nn as nn

from collections import defaultdict
import torch.optim as optim

from utils.train_helper import model_snapshot, load_model
from utils.logger import get_logger

from torch_geometric.loader import DataLoader
from models.DimeNet.model import DimeNet
from models.SphereNet.model import SphereNet
from models.ComENet.model import ComENet
from dataset.dataset_loader import get_dataset, get_qm9_dataset

In [3]:
_, _, test = get_qm9_dataset('../data/qm9_hackerthon')

Load Dataset
Load Dataset


In [4]:
test_dataset = DataLoader(test, batch_size=64)

In [5]:
# config_file = glob('../config/QM9/DimeNet/config.yaml')[0]
config_file = glob('../../exp/DimeNet/__0922_175635/config.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [6]:
best_model = DimeNet(config.model)

In [12]:
best_model_dir = os.path.join('../../exp/DimeNet/__0922_175635/model_save', 'best.pth')
best_snapshot = load_model(best_model_dir)
best_model.load_state_dict(best_snapshot)

<All keys matched successfully>

In [13]:
best_model = best_model.to(device=0)

In [14]:
best_model.eval()

submission = []

for data_batch in tqdm(test_dataset):
    data_batch = data_batch.to(device=0)
    
    with torch.no_grad():
        out = best_model(z=data_batch.z, pos=data_batch.pos, edge_index=data_batch.edge_index,
                         batch=data_batch.batch)
        
        pred = out.cpu().detach().numpy().reshape(-1).tolist()
        submission += pred

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:24<00:00, 21.28it/s]


In [15]:
submission = np.array(submission)

In [16]:
submission = pd.DataFrame(submission)

In [25]:
np.savetxt('sample1.csv', submission)